In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats,
                                    leaguedashplayerptshot)
import time

In [2]:
current_year="2023-24"

In [3]:
pbp2023=pd.read_csv(filepath_or_buffer="Data/"+current_year+"_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [4]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season=current_year,season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season=current_year,
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [5]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [6]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
247,Jaylin Williams,0.840000
388,Moritz Wagner,0.776842
46,Brandin Podziemski,0.766265
168,Garrison Mathews,0.685714
192,Isaiah Joe,0.536170
309,Kenrich Williams,0.524503
495,Tim Hardaway Jr.,0.524503
216,Jalen Brunson,0.524370
384,Miles McBride,0.504000
390,Moses Moody,0.480000


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
503,Trendon Watford,1.861017
285,Josh Okogie,1.770492
273,Jordan Goodwin,1.688276
527,Xavier Tillman,1.642718
480,T.J. McConnell,1.567059
114,Day'Ron Sharpe,1.462500
454,Russell Westbrook,1.455652
75,Chris Duarte,1.390909
434,Paul Reed,1.372500
29,Ausar Thompson,1.363404


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
371,Matisse Thybulle,5.170044
328,Kris Dunn,4.525714
12,Alex Caruso,4.486364
153,Dyson Daniels,4.475229
384,Miles McBride,4.320000
117,De'Anthony Melton,4.008247
434,Paul Reed,3.825000
470,Shai Gilgeous-Alexander,3.715116
480,T.J. McConnell,3.684706
19,Andre Drummond,3.645283


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [9]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
319,Kevon Looney,15.360000
509,Tristan Thompson,14.341935
102,Daniel Gafford,14.029008
114,Day'Ron Sharpe,13.747500
139,Domantas Sabonis,13.352542
151,Dwight Powell,13.292308
500,Trayce Jackson-Davis,13.225532
297,Jusuf Nurkic,12.994161
402,Nick Richards,12.793625
240,Jarrett Allen,12.235294


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [10]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [11]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [12]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
169,Gary Harris,0.232353,0.005882,0.008824,0.532353,0.014706,0.269118,1.063235
355,Malcolm Brogdon,0.232353,0.076471,0.261765,0.191176,0.152941,0.211765,1.126471
325,Klay Thompson,0.232353,0.361765,0.194118,0.141176,0.129412,0.079412,1.138235
354,Malaki Branham,0.232353,0.179412,0.297059,0.041176,0.023529,0.502941,1.276471
379,Michael Porter Jr.,0.232353,0.502941,0.167647,0.088235,0.326471,0.073529,1.391176
35,Bennedict Mathurin,0.232353,0.300000,0.111765,0.138235,0.482353,0.220588,1.485294
458,Saddiq Bey,0.232353,0.279412,0.038235,0.320588,0.188235,0.429412,1.488235
218,Jalen Green,0.497059,0.311765,0.220588,0.176471,0.158824,0.129412,1.494118
171,Gary Trent Jr.,0.232353,0.014706,0.005882,0.632353,0.120588,0.511765,1.517647
476,Spencer Dinwiddie,0.232353,0.405882,0.423529,0.164706,0.144118,0.150000,1.520588


In [13]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
430,Patrick Beverley,0.908824,0.158824,0.358824,0.585294,0.038235,0.683824,2.733824
454,Russell Westbrook,0.879412,0.211765,0.041176,0.811765,0.067647,0.955882,2.967647


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [14]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
247,Jaylin Williams,1.000000,0.900000,0.864706,0.738235,0.964706,0.791176,5.258824
388,Moritz Wagner,0.997059,0.802941,0.676471,0.697059,0.961765,0.770588,4.905882
19,Andre Drummond,0.741176,0.794118,0.944118,0.944118,0.752941,0.714706,4.891176
212,Jae'Sean Tate,0.955882,0.638235,0.950000,0.614706,0.747059,0.964706,4.870588
185,Herbert Jones,0.917647,0.622059,0.985294,0.844118,0.655882,0.741176,4.766176
264,John Konchar,0.902941,0.673529,0.844118,0.852941,0.455882,0.891176,4.620588
461,Sandro Mamukelashvili,0.232353,0.838235,0.832353,0.676471,0.985294,0.970588,4.535294
369,Marvin Bagley III,0.958824,0.891176,0.982353,0.129412,0.952941,0.611765,4.526471
434,Paul Reed,0.232353,0.938235,0.629412,0.967647,0.873529,0.882353,4.523529
84,Cody Zeller,0.973529,0.879412,0.426471,0.476471,0.997059,0.750000,4.502941


# The Trickshot Grenadier Award (presented by Dude Perfect)

Highest sum of percentile ranks in FGA, FGA frequency & eFG% on shots with 4 seconds or less on the shotclock (minimum 50th percentile in FGA) (credit to BehavioralSink & Bylanta for the idea)

In [15]:
late_shotclock_shots=leaguedashplayerptshot.LeagueDashPlayerPtShot(shot_clock_range_nullable="4-0 Very Late").league_dash_ptshots.get_data_frame().loc[:, 'PLAYER_ID':'EFG_PCT']
late_shotclock_shots["FGA_percentile"]=late_shotclock_shots.FGA.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile=late_shotclock_shots.query("FGA_percentile>=0.5").copy()
late_shotclock_shots_top_50_percentile["FGA_percentile_among_top_50"]=late_shotclock_shots_top_50_percentile.FGA.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["FGA_FREQ_percentile"]=late_shotclock_shots_top_50_percentile.FGA_FREQUENCY.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["EFG_percentile"]=late_shotclock_shots_top_50_percentile.EFG_PCT.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["avg_percentile"]=late_shotclock_shots_top_50_percentile[['FGA_FREQ_percentile','FGA_percentile','EFG_percentile']].sum(axis=1)
late_shotclock_shots_top_50_percentile.nlargest(n=10,columns="avg_percentile")

,PLAYER_ID,PLAYER_NAME,PLAYER_LAST_TEAM_ID,PLAYER_LAST_TEAM_ABBREVIATION,AGE,GP,G,FGA_FREQUENCY,FGM,FGA,FG_PCT,EFG_PCT,FGA_percentile,FGA_percentile_among_top_50,FGA_FREQ_percentile,EFG_percentile,avg_percentile
2,203999,Nikola Jokic,1610612743,DEN,28.0,46,38,0.123,49,99,0.495,0.540,0.995816,0.991342,0.870130,0.839827,2.705773
5,202695,Kawhi Leonard,1610612746,LAC,32.0,42,37,0.116,38,83,0.458,0.542,0.989540,0.978355,0.818182,0.844156,2.651877
106,1627936,Alex Caruso,1610612741,CHI,29.0,42,22,0.114,21,35,0.600,0.800,0.771967,0.528139,0.809524,1.000000,2.581490
44,1627747,Caris LeVert,1610612739,CLE,29.0,35,27,0.119,23,55,0.418,0.527,0.905858,0.805195,0.844156,0.822511,2.572524
23,202704,Reggie Jackson,1610612743,DEN,33.0,48,37,0.150,28,68,0.412,0.478,0.949791,0.896104,0.939394,0.679654,2.568838
13,202331,Paul George,1610612746,LAC,33.0,43,37,0.098,33,72,0.458,0.576,0.970711,0.939394,0.666667,0.900433,2.537811
85,200768,Kyle Lowry,1610612766,CHA,37.0,36,21,0.167,15,39,0.385,0.487,0.815900,0.619048,0.956710,0.727273,2.499882
210,201580,JaVale McGee,1610612758,SAC,36.0,36,15,0.180,13,20,0.650,0.650,0.548117,0.064935,0.987013,0.961039,2.496169
154,1626174,Christian Wood,1610612747,LAL,28.0,42,19,0.117,16,26,0.615,0.673,0.665272,0.307359,0.831169,0.982684,2.479125
37,202710,Jimmy Butler,1610612748,MIA,34.0,33,28,0.128,24,57,0.421,0.474,0.920502,0.835498,0.883117,0.658009,2.461628


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked where the blocker is at least 5 inches shorter than the blockee

In [16]:
blocks=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
66,Fred VanVleet,28
54,Derrick White,13
127,Luguentz Dort,13
23,CJ McCollum,11
60,Donovan Mitchell,9
155,Patrick Beverley,9
168,Scottie Barnes,9
40,D'Angelo Russell,8
70,Grayson Allen,8
110,Jrue Holiday,8


# The "Pick On Someone Your Own Size" Award

most shots blocked where the blocker is at least 5 inches taller than the blockee

In [17]:
blocking_player_heights.query('height_difference <= -5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
186,Victor Wembanyama,108
22,Brook Lopez,84
27,Chet Holmgren,74
187,Walker Kessler,59
165,Rudy Gobert,53
121,Kristaps Porzingis,45
10,Anthony Davis,41
146,Nic Claxton,39
50,Dereck Lively II,35
82,Jakob Poeltl,35


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award (presented by Carmelo Anthony)

Lowest contested rebound percentage, minimum 50% of games played

In [18]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
99,Tyus Jones,0.044
439,Marcus Sasser,0.048
462,Jacob Gilyard,0.065
60,CJ McCollum,0.079
216,Trae Young,0.083
192,Aaron Holiday,0.084
356,Kessler Edwards,0.091
249,Tyler Herro,0.093
106,Cameron Payne,0.096
58,Justin Holiday,0.097


alternatively: restricting to players > 6 foot 6 inches in height

In [19]:
above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
101,Kessler Edwards,0.091
50,Furkan Korkmaz,0.150
69,Jalen McDaniels,0.167
49,Franz Wagner,0.168
151,Sam Hauser,0.181
84,Joe Ingles,0.184
77,Jayson Tatum,0.189
54,Gordon Hayward,0.195
7,Anthony Black,0.196
30,Dalen Terry,0.196


# The "Glass Cleaner" Award (presented by Dennis Rodman, sponsored by Windex)

Highest contested rebound percentage, minimum 50% of games played (https://www.nba.com/stats/players/rebounding?PerMode=Totals&dir=D&sort=REB_CONTEST_PCT)

In [20]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
301,Zeke Nnaji,0.581
170,Luke Kornet,0.547
172,Daniel Theis,0.545
352,Day'Ron Sharpe,0.521
52,Andre Drummond,0.519
177,Marvin Bagley III,0.516
162,Isaiah Hartenstein,0.512
422,Walker Kessler,0.507
303,Paul Reed,0.502
257,Daniel Gafford,0.497


alternatively: restricting to players < 6 foot 7 inches in height

In [21]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
96,Josh Okogie,0.434
103,Kenrich Williams,0.415
41,Danuel House Jr.,0.400
167,Zion Williamson,0.383
76,Jaden Springer,0.372
77,Jae'Sean Tate,0.368
154,Terance Mann,0.363
101,KJ Martin,0.355
133,Ochai Agbaji,0.350
127,Moses Moody,0.340
